# Sanic 配置简介

Sanic 在应用程序对象的 `config` 属性中持有配置。配置对象只是使用点表示法或像字典一样修改的对象。

```python
app = Sanic("myapp")
app.config.DB_NAME = "appdb"
app.config["DB_USER"] = "appuser"
```

你也可以像在普通字典上一样使用 {meth}`update` 方法：

```python
db_settings = {
    'DB_HOST': 'localhost',
    'DB_NAME': 'appdb',
    'DB_USER': 'appuser'
}
app.config.update(db_settings)
```

```{note}
在 Sanic 中，将配置值命名为大写字母是标准做法。实际上，如果你开始混合使用大写和小写名称，可能会遇到奇怪的行为。
```

## 载入配置

## 环境变量配置 Sanic

任何以 `SANIC_` 前缀定义的环境变量都将应用到 Sanic 配置中。例如，设置 `SANIC_REQUEST_TIMEOUT` 将由应用程序自动加载并输入到  `REQUEST_TIMEOUT` 配置变量中。

```bash
$ export SANIC_REQUEST_TIMEOUT=10
```

```bash
>>> print(app.config.REQUEST_TIMEOUT)
10
```

你可以更改 Sanic 在启动时期望的前缀。

```bash
$ export MYAPP_REQUEST_TIMEOUT=10
>>> app = Sanic(__name__, env_prefix='MYAPP_')
>>> print(app.config.REQUEST_TIMEOUT)
10
```


你还可以完全禁用环境变量加载。

```python
app = Sanic(__name__, load_env=False)
```

## 使用 `app.update_config`

Sanic 实例有非常通用的方法用于加载配置：`app.update_config`。你可以向它提供文件路径、字典、类，或者其他任何类型的对象。

### 从文件中加载配置

假设你有名为 `my_config.py` 的文件，它看起来是这样的。

```python
# my_config.py
A = 1
B = 2
```

你可以通过将文件路径传递给 `app.update_config` 来将其作为配置值加载。

```bash
>>> app.update_config("/path/to/my_config.py")
>>> print(app.config.A)
1
```

这个路径也接受 bash 风格的环境变量。

```bash
$ export my_path="/path/to"
>>> app.update_config("${my_path}/my_config.py")
```

```{note}
只需记住，你必须以 `${environment_variable}` 的格式提供环境变量，而 `$environment_variable` 不会被展开（被视为“普通”文本）。
```

### 从 dict 或者 class 加载配置

`app.update_config` 也支持 普通的 dict 对象：

```python
app.update_config({"A": 1, "B": 2})
```

你可以定义自己的配置类，并将其传递给 `app.update_config`。

```python
class MyConfig:
    A = 1
    B = 2

app.update_config(MyConfig)
```

它甚至可以被实例化：

```bash
app.update_config(MyConfig())
```

## 类型 cast

当从环境变量加载时，Sanic 会尝试将值转换为预期的 Python 类型。这特别适用于：`int`，`float` 和 `bool`。

关于布尔值，允许以下不区分大小写的值：

- `True`: y, yes, yep, yup, t, true, on, enable, enabled, 1
- `False`: n, no, f, false, off, disable, disabled, 0

如果值无法转换，它将默认为 `str`。

此外，Sanic 可以通过使用额外的类型转换器来配置以转换其他类型。这应该是任何可调用的，返回值或引发 `ValueError`。

```python
app = Sanic(..., config=Config(converters=[UUID]))
```